In [16]:
import shutil
import kagglehub
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_process import ArmaProcess

### About Dataset

**Context**

Bitcoin is the longest running and most well known cryptocurrency, first released as open source in 2009 by the anonymous Satoshi Nakamoto. Bitcoin serves as a decentralized medium of digital exchange, with transactions verified and recorded in a public distributed ledger (the blockchain) without the need for a trusted record keeping authority or central intermediary. Transaction blocks contain a SHA-256 cryptographic hash of previous transaction blocks, and are thus "chained" together, serving as an immutable record of all transactions that have ever occurred. As with any currency/commodity on the market, bitcoin trading and financial instruments soon followed public adoption of bitcoin and continue to grow. Included here is historical bitcoin market data at 1-min intervals for select bitcoin exchanges where trading takes place. Happy (data) mining! 

Content
([See](https://github.com/mczielinski/kaggle-bitcoin/) for automation/scraping script) \
**btcusd_1-min_data.csv**

CSV files for select bitcoin exchanges for the time period of Jan 2012 to Present (Measured by UTC day), with minute to minute updates of OHLC (Open, High, Low, Close) and Volume in BTC.

If a timestamp is missing, or if there are jumps, this may be because the exchange (or its API) was down, the exchange (or its API) did not exist, or some other unforeseen technical error in data reporting or gathering. I'm not perfect, and I'm also busy! All effort has been made to deduplicate entries and verify the contents are correct and complete to the best of my ability, but obviously trust at your own risk.
Acknowledgements and Inspiration

Bitcoin charts for the data, originally. Now thank you to the Bitstamp API directly. The various exchange APIs, for making it difficult or unintuitive enough to get OHLC and volume data at 1-min intervals that I set out on this data scraping project. Satoshi Nakamoto and the novel core concept of the blockchain, as well as its first execution via the bitcoin protocol. I'd also like to thank viewers like you! Can't wait to see what code or insights you all have to share. 

[DatasetLink](https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data)

Column Descriptions
| Column Name               | Description                                           |
|---------------------------|-------------------------------------------------------|
| Timestamp 	            | Start time of time window (60s window), in Unix time  |
| Open 	                    | Open price at start time window                       |
| High       	            | High price at start time window                       |
| Low            	        | Low price at start time window                        |     
| Close              	    | Close price at start time window                      |
| Volume 	                | Volume of BTC transacted in this window               |
| datetime 	                | date & time                                           |

In [8]:
# Download latest version
path = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")

print("Path to dataset files:", path)

100%|██████████| 112M/112M [00:11<00:00, 10.6MB/s] 

Extracting files...


Path to dataset files: /home/alex/.cache/kagglehub/datasets/mczielinski/bitcoin-historical-data/versions/185


In [ ]:
full_path = f'{path}/btcusd_1-min_data.csv'

In [ ]:
shutil.copy(full_path, './')

'./btcusd_1-min_data.csv'

In [3]:
df = pd.read_csv('btcusd_1-min_data.csv')
df.head()

,Timestamp,Open,High,Low,Close,Volume,datetime
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:01:00+00:00
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:02:00+00:00
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:03:00+00:00
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:04:00+00:00
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:05:00+00:00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6944800 entries, 0 to 6944799
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Timestamp  float64
 1   Open       float64
 2   High       float64
 3   Low        float64
 4   Close      float64
 5   Volume     float64
 6   datetime   object 
dtypes: float64(6), object(1)
memory usage: 370.9+ MB


In [7]:
missing_rows = df[df.isna().any(axis=1)]
print(missing_rows)

            Timestamp      Open      High       Low     Close    Volume  \
443279   1.352009e+09     10.49     10.49     10.49     10.49  0.000000   
443280   1.352009e+09     10.49     10.49     10.49     10.49  0.000000   
443281   1.352009e+09     10.49     10.49     10.49     10.49  0.000000   
443282   1.352009e+09     10.49     10.49     10.49     10.49  0.000000   
443283   1.352009e+09     10.49     10.49     10.49     10.49  0.260338   
...               ...       ...       ...       ...       ...       ...   
6944795  1.742169e+09  82554.00  82554.00  82554.00  82554.00  0.072000   
6944796  1.742169e+09  82584.00  82615.00  82584.00  82615.00  0.194670   
6944797  1.742169e+09  82555.00  82555.00  82555.00  82555.00  0.002680   
6944798  1.742170e+09  82555.00  82555.00  82555.00  82555.00  0.000000   
6944799  1.742170e+09  82569.00  82569.00  82566.00  82566.00  0.003019   

        datetime  
443279       NaN  
443280       NaN  
443281       NaN  
443282       NaN  
4432

In [8]:
columns_with_nan = df.columns[df.isna().any()].tolist()
print(columns_with_nan)

['datetime']


In [9]:
df = df.dropna()
df = df.reset_index(drop=True)
print(df.isna().sum())

Timestamp    0
Open         0
High         0
Low          0
Close        0
Volume       0
datetime     0
dtype: int64


In [10]:
if df['datetime'].astype(str).str.endswith('+00:00').all():
    df['datetime'] = df['datetime'].str[:-6]
else:
    print('None')

In [11]:
df["datetime"] = pd.to_datetime(df["datetime"])

In [12]:
df['datetime']

0         2012-01-01 10:01:00
1         2012-01-01 10:02:00
2         2012-01-01 10:03:00
3         2012-01-01 10:04:00
4         2012-01-01 10:05:00
                  ...        
6782275   2025-03-14 23:56:00
6782276   2025-03-14 23:57:00
6782277   2025-03-14 23:58:00
6782278   2025-03-14 23:59:00
6782279   2025-03-15 00:00:00
Name: datetime, Length: 6782280, dtype: datetime64[ns]

In [13]:
def generate_timestamps(n, start=1700000000, freq=60):
    return np.arange(start, start + n * freq, freq)

def generate_price_series(n, start_price=100.0, ar=[1, -0.5], ma=[1], sigma=2.0):
    ar_params = np.r_[1, -np.array(ar)]
    ma_params = np.r_[1, ma]
    arma_process = ArmaProcess(ar_params, ma_params)
    return start_price + np.cumsum(arma_process.generate_sample(nsample=n, scale=sigma))

n = 100000

timestamps = generate_timestamps(n)
open_prices = generate_price_series(n)
high_prices = open_prices + np.abs(np.random.normal(2, 0.5, n))
low_prices = open_prices - np.abs(np.random.normal(2, 0.5, n))
close_prices = generate_price_series(n, start_price=open_prices[-1])
volume = np.random.randint(100, 10000, n)
datetime_values = pd.to_datetime(timestamps, unit='s')

df_synthetic = pd.DataFrame({
    'Timestamp': timestamps,
    'Open': open_prices,
    'High': high_prices,
    'Low': low_prices,
    'Close': close_prices,
    'Volume': volume,
    'datetime': datetime_values
})

In [14]:
df_synthetic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Timestamp  100000 non-null  int64         
 1   Open       100000 non-null  float64       
 2   High       100000 non-null  float64       
 3   Low        100000 non-null  float64       
 4   Close      100000 non-null  float64       
 5   Volume     100000 non-null  int64         
 6   datetime   100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 5.3 MB


In [15]:
df_synthetic.head()

,Timestamp,Open,High,Low,Close,Volume,datetime
0,1700000000,96.549333,99.099021,94.686044,1241.758948,4059,2023-11-14 22:13:20
1,1700000060,85.761955,87.634180,83.826519,1241.833346,5715,2023-11-14 22:14:20
2,1700000120,71.400127,73.881590,69.919560,1240.769445,3469,2023-11-14 22:15:20
3,1700000180,61.633888,63.340438,59.879367,1238.191881,9760,2023-11-14 22:16:20
4,1700000240,58.696808,60.787739,57.272100,1234.310779,3647,2023-11-14 22:17:20
